__Unit Test__

The purpose of the notebook is to show the safety features in the data_store library that will accomodate all corner cases. 
Let's start with the imports:

In [20]:
from data_store import data_store
import time

- Creating an object for data_store. A database named 'database.json' has been created for me in my working directory as I have not provided a target directory.  

In [21]:
dataparser = data_store()

Database created in directory, C:\Users\Lenovo\Documents\fw_data_store/database.json


- Values are created with the format , obj.Create(key,value,Time-to-Live). 
- If no 'Time-to-live'(TTL) is provided, the value defaults to None.

In [22]:
dataparser.Create('Dwight',100,5)
dataparser.Create('Jim',500)
dataparser.Create('Michael',1000)

Value added
Value added
Value added


In [23]:
dataparser.Read('Jim')

'500'

- Read() returns the value of a valid key in the database. Let's see how it returns values with a TTL.

In [24]:
print('Before the wait')
val = dataparser.Read('Dwight')
print(val)
time.sleep(5)
print('After the wait')
val = dataparser.Read('Dwight')


Before the wait


Exception: Key's Time-To-Live has expired. Unable to read.

- Create() also prohibits overwriting existing keys

In [25]:
dataparser.Create('Jim','Jim two')

Exception: Key is already present.

- Create() also blocks keys that are over 32 characters long or are not strings

In [26]:
dataparser.Create('blah' * 100, 100)

Exception: Key size is capped at 32. Given key length is 400

- The Delete() function pops a key from the database if it's present. 

In [27]:
dataparser.Delete('Jim')

Key-value pair deleted


In [28]:
dataparser.Read('Jim')

Exception: Key not found in database

- Let's try deleting the key again

In [29]:
dataparser.Delete('Jim')

Exception: Key not found in database

 - DisplayAll() prints the saved data in the form of a dictionary with key-value pairs

In [30]:
dataparser.DisplayAll()

{'Dwight': 100, 'Michael': 1000}


- ClearAll() clears the database (after gettting confirmation of course)

In [33]:
dataparser.ClearAll()

Are you sure you want to clear the database? (y/n) y


Data cleared


In [34]:
dataparser.DisplayAll()

Exception: Unable to display. Database is empty

- Let's try some corner cases where some data is not provided properly

In [35]:
#Create()
#1) No key 
dataparser.Create(value = 25,ttl =6000)



Exception: No key was provided.

In [36]:
#2) No Value
dataparser.Create(key='Pam',ttl =6000)

Value added


- Here, the missing value will default to the JSON object, NULL.

In [37]:
#3)Incorrect Data Type

dataparser.Create(key = ['Angela,Oscar'], value = 3313)

Exception: Key value is not a string. Inputed key is of type: <class 'list'>

- Create() also ensures the data file size is below 1 GB and that the memory size of key values dont exceed 16 KB

- The program is also __thread-safe__. It prevents race condition by using thread locks and is suitable for multi-threadin, as tried here- 

In [38]:
import concurrent.futures

with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for i in range(3):
        executor.submit(dataparser.Create('Kevin',500))
        time.sleep(2)
        executor.submit(dataparser.Delete('Kevin'))

Value added
Key-value pair deleted
Value added
Key-value pair deleted
Value added
Key-value pair deleted


In [39]:
dataparser.DisplayAll()

{'Pam': None}


- Since the key 'Kevin' makes no appearance in the data, it's understood that the key was created and destroyed sequentially. 

In [41]:
#Read()

dataparser.Read('Darth Vader')

Exception: Key not found in database

In [42]:
dataparser.Read(56)

Exception: Key value is not a string. Inputed key is of type: <class 'int'>

In [44]:
dataparser.Delete({'Jim':'Jimothy'})

Exception: Key value is not a string. Inputed key is of type: <class 'dict'>